# **Exercício de Fixação de Conceitos 2 - Questão 3**

### **Enunciado**:
- Tomando o mesmo problema de classificação de dados da base MNIST, use o *framework* Keras, tendo o TensorFlow como *backend* e realize o treinamento de uma rede neural MLP.
- Busque inspiração em resultados já publicados na literatura e/ou adote o procedimento de tentativa e erro para definir, da melhor forma que você puder:
    - O número da camadas intermediárias.
    - O número de neurônios por camada.
    - O algoritmo de ajuste de pesos.
    - A taxa de *Dropout* (onde for pertinente).
    - O número de épocas de treinamento.
- Procure trabalhar com a média de várias execuções (junto a cada configuração candidata) para se chegar a um índice de desempenho mais estável.
- Um código que pode servir de ponto de partida é fornecido a seguir, considerando 1 camada intermediária com 512 neurônios, algoritmo de treinamento ADAM, ocorrência de *dropout* numa taxa de 50%, 5 épocas de treinamento e função de perda sendo uma forma de entropia cruzada.
- A sua proposta deve ser capaz de superar o desempenho dessa sujestão abaixo e você deve descrever de forma objetiva o caminho trilhado até a sua configuração final de código para a rede neural MLP, assim como uma comparação de desempenho com a sugestão abaixo.


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

2.2.0-rc3
2.3.0-tf


In [2]:
# Download do dataset e normalização das amostras de entrada de treinamento e teste:

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train/255.0, x_test/255.0

11493376/11490434 [==============================] - 0s 0us/step


### **Solução do exercício:**
- O objetivo desta atividade é propor uma rede MLP alternativa que apresente um desempenho superior àquela do enunciado.
- Como discutido nas aulas de dúvida, estamos interessados em analisar o desempenho das redes MLP (com diferentes configurações de hiperparâmetros) apenas com relação aos dados de treinamento. Deste modo, não é necessário separar uma parcela das 60.000 amostras para validação dos modelos.
- São treinadas 5 redes MLP para cada configuração de hiperparâmetros, sendo o desempenho final tomado como a média dos desempenhos das 5 redes obtidas após o treinamento.
- As métricas de desempenho investigadas são as perdas (*loss*) e a acurácia.


**a) Arquitetura e treinamento propostos no enunciado:**
- Arquitetura:
  - Uma camada intermediária com 512 neurônios com funções de ativação ReLU e taxa de ocorrência de *dropout* de 50%.
  - Uma camada de saída com 10 neurônios com função de ativação softmax (não deve ser alterada).
- Treinamento: 
  - Algoritmo adaptativo Adam, 5 épocas e 32 amostras por mini-batch (default do método fit()).
  - Função custo (loss): sparse_categorical_crossentropy (não deve ser alterada).
  - Métrica auxiliar: Acurácia (não deve ser alterada).
- Resultado do treinamento dos 5 modelos:
  - Perdas: 0.0812
  - Acurácia: 0.9744

**b) Impacto de diferentes funções de ativação dos neurônios da camada intermediária:**
- O primeiro hiperparâmetro investigado é o tipo de função de ativação dos neurônios da camada intermediária.
- Serão considerados as seguintes funções de ativação: sigmoide, tangente hiperbólica, ReLU e LeakyReLU.
- A Tabela 1 apresenta o desempenho para cada uma das funções de ativação. Como pode ser observado, o desempenho da rede piorou tanto para as funções sigmoidais quanto para a LeakyReLU. 
- Deste modo, a ReLU será mantida como função de ativação dos neurônios da rede.

<h><center>Tabela 1: Métricas de desempenho para cada tipo de função de ativação.</center></h>

| Função de ativação | Perdas | Acurácia |
|--------------|--------|----------|
|Sigmoide    |0.1060  |0.9680    |
|tanh        |0.1364  |0.9583    |
|ReLU        |0.0812  |0.9744    |
|Leaky ReLU  |0.1755  |0.9485    |



**c) Impacto de diferentes algoritmos de otimização:**
- São considerados os algoritmos de otimização SGD, SGD+momentum, NAG, Adagrad, Adadelta, Adam.
- Como pode ser observado na Tabela 2, a rede treinada com Adam atingiu um desempenho superior que as redes treinadas com os demais algoritmos.
- Desta forma, o Adam será mantido para as próximas investigações.

<h><center>Tabela 2: Métricas de desempenho em função do algoritmo de otimização.</center></h>

| Algoritmo de otimização | Perdas | Acurácia |
|--------------|--------|----------|
|SGD    |0.2571  |0.9266    |
|SGD + Momentum        |0.0967  |0.9706    |
|NAG        |0.0961  |0.9711    |
|Adagrad  |0.4596  |0.8691    |
|Adadelta  |1.5383  |0.5737    |
|Adam  |0.0812  |0.9744    |

**d) Impacto do número de neurônios da camada intermediária:**
- Agora passamos a analisar o impacto do número de neurônios da camada intermediária.
- Pode-se observar na Tabela 3 que a rede com 100 neurônios apresentou um desempenho bem inferior as demais. 
- Já as redes com mais neurônios que aquela proposta no enunciado não apresentaram melhoras significativas no desempenho.
- Por essa razão e por possui um menor número de parâmetros ajustáveis, continuamos com a rede neural de 512 neurônios nas próximas análises.

<h><center>Tabela 3: Métricas de desempenho em função do número de neurônios da camada intermediária.</center></h> 

| Nº Neurônios | Perdas | Acurácia |
|--------------|--------|----------|
|100           |0.1756  |0.9472    |  
|300           |0.0956  |0.9701    |
|512           |0.0812  |0.9744    | 
|700           |0.0775  |0.9754    | 
|900           |0.0751  |0.9765    | 

**e) Impacto da taxa de ocorrência de dropout da camada intermediária no desempenho da rede:**
- O último hiperparâmetro que falta ser modificado na camada intermediária é a taxa de ocorrência de dropout.
- Sabemos que o dropout é uma técnica de regularização e, consequentemente, tenta evitar que a rede sofra overfitting. 
- Contudo, como não estamos preocupados em evitar overfitting e aumentar a capacidade de generalização, reduzir a taxa de ocorrência de dropout leva ao aumento de desempenho da rede, como pode ser verificado na Tabela 4.
- Deste modo, para obter um modelo que supera o desempenho da rede neural proposta no enunciado, basta utilizar uma taxa de dropout inferior a 0.5.

<h><center>Tabela 4: Métricas de desempenho em função da taxa de ocorrência de dropout na camada intermediária.</center></h>

| Taxa de dropout | Perdas | Acurácia |
|-----------------|--------|----------|
|0.1              |0.0350  |0.9887    |
|0.3              |0.0531  |0.9829    |
|0.5              |0.0812  |0.9744    |
|0.7              |0.1402  |0.9575    |
|0.9              |0.3811  |0.8847    |

**f) Arquitetura que supera a proposta do enunciado:**
- Arquitetura:
  - Uma camada intermediária com 512 neurônios com funções de ativação ReLU e taxa de ocorrência de *dropout* de **10%**.
  - Uma camada de saída com 10 neurônios com função de ativação softmax (não deve ser alterada).
- Treinamento: 
  - Algoritmo adaptativo Adam, 5 épocas e 32 amostras por mini-batch (default do método fit()).
  - Função custo (loss): sparse_categorical_crossentropy (não deve ser alterada).
  - Métrica auxiliar: Acurácia (não deve ser alterada).
- Resultado do treinamento dos 5 modelos:
  - Perdas: 0.0350
  - Acurácia: 0.9887

In [0]:
num_models = 5
EPOCHS = 5
media_metricas = []
rates = [0.5, 0.1]

for rate in rates:
    
    lista_metricas = { 'loss': [], 'accuracy': []}
    metricas = {}
    
    print(f'Rede Neural com taxa de dropout de {rate*100}%:')

    for i in range(0, num_models):

        model = keras.models.Sequential([
            keras.layers.Flatten(input_shape=x_train.shape[1:]),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dropout(rate),
            keras.layers.Dense(10, activation='softmax')
        ])

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(x_train, y_train, epochs=5, verbose=False)
        
        lista_metricas['loss'].append(history.history['loss'][-1])
        lista_metricas['accuracy'].append(history.history['accuracy'][-1])

    for key in lista_metricas.keys():
        metricas[key] = sum(lista_metricas[key])/len(lista_metricas[key])

    media_metricas.append(metricas)
    print(media_metricas)

# **Exercício de Fixação de Conceitos 2 - Questão 4**

### **Enunciado**:
- Tomando o mesmo problema de classificação de dados da base MNIST e novamente usando o *framework* Keras, tendo o TensorFlow como *backend*, realize o treinamento de uma rede neural com camadas convolucionais, usando *maxpooling* e *dropout*.
- Mais uma vez, é apresentada a seguir uma sugestão de código e de configuração de hiperparâmetros que pode ser tomada como ponto de partida.
- A sua proposta deve superar, em termos de desempenho médio, essa sugestão fornecida abaixo.
- Descreva de forma objetiva o caminho trilhado até sua configuração final de código.
- Compare os resultados (em termos de taxa de acerto de classificação) com aqueles obtidos pelos três tipos de máquinas de aprendizado adotadas nas atividades anteriores (classificador linear, ELM e MLP).


In [0]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

### **Parte 1 - Solução do exercício:**
- Diferente do que foi feito na questão 2, onde treinamos 5 redes neurais para cada configuração de hiperparâmetros, aqui são treinadas apenas 3 redes, pois o tempo de treinamento de uma rede convolucional é muito maior que de uma simples rede MLP.

**a) Arquitetura e treinamento propostos no enunciado:**
- Arquitetura:
  - Uma camada convolucional com 32 kernels de dimensão 3x3, stride (1,1) e com funções de ativação ReLU.
  - Uma segunda camada convolucional com 64 kernels de dimensão 3x3, stride (1,1) e com funções de ativação ReLU, MaxPooling de dimensão 2x2 e taxa de ocorrência de dropout de 25%.
  - Camada do tipo Flatten
  - Camada fully-connected com 128 neurônios com função de ativação ReLU e taxa de ocorrência de dropout de 50%.
  - Uma camada de saída com 10 neurônios com função de ativação softmax (não deve ser alterada).
- Treinamento: 
  - Algoritmo adaptativo Adam, 5 épocas e 32 amostras por mini-batch (default do método fit()).
  - Função custo (loss): sparse_categorical_crossentropy (não deve ser alterada).
  - Métrica auxiliar: Acurácia (não deve ser alterada).
- Resultado do treinamento dos 3 modelos:
  - Perdas: 0.0434
  - Acurácia: 0.9866

**b) Alterações na rede convolucional proposta e seus impactos no desempenho:**

- A primeira etapa das alterações foi relacionada aos hiperparâmetros da primeira camada convolucional. Foram realizadas as seguintes modificações:
  - 1) Redução de 32 para 16 kernels
  - 2) Aumento de 32 para 64 kernels
  - 3) Redução do tamanho dos kernels de 3x3 para 2x2
  - 4) Aumento do tamanho dos kernels de 3x3 para 4x4
  - 5) Aumento do stride de (1, 1) para (2, 2)
  - 6) Inserção de uma camada de Max Pooling
- A Tabela 5 apresenta as métricas de desempenho, perdas e acurácia, para as redes com cada uma das alterações acima.

<h><center>Tabela 5: Métricas de desempenho para cada uma das alterações de hiperparâmetros realizadas na primeira camada convolucional.</center></h>

| Alteração | Perdas | Acurácia |
|-----------------|--------|----------|
|1              |0.0421  |0.9871    |
|2              |0.0439  |0.9864    |
|3              |0.0464  |0.9855    |
|4              |0.0409  |0.9873    |
|5              |0.0489  |0.9850    |
|6              |0.0519  |0.9841    |

- Como pode ser observado na Tabela 5, duas alterações levaram a um aumento no desempenho: redução do número de kernels de 32 para 16 e o aumento das dimensões de cada kernel de 3x3 para 4x4.
- Diante disso, foram treinadas redes com essas duas alterações juntas visando alcançar um desempenho ainda maior. O resultado dessa investigação está apresentado na Tabela 6. Nota-se que o desempenho dessa configuração foi menor do que o desempenho das redes com cada uma das alterações feitas individualmente.
- A segunda parte das alterações foram feitas considerando a primeira camada convolucional com 32 kernels de dimensão 4x4. As modificações, cujo os desempenhos estão apresentados na Tabela 6, foram:
  - 7) Remoção da camada de Max Pooling da segunda camada convolucional
  - 8) Remoção da camada fully-connected.  

<h><center>Tabela 6: Métricas de desempenho para a segunda parte de alterações realizadas na rede convolucional.</center></h>

| Alteração | Perdas | Acurácia |
|---------------|--------|----------|
|1+4            |0.0430  |0.9869    |
|7              |0.0342  |0.9894    |
|8              |0.0239  |0.9924    |
|7+8            |0.0158  |0.9949    |

- Pode-se observar que as modificações 7 e 8 levaram a um aumento substancial no desempenho da rede, atingindo uma acurácia de 99,49% com as duas alterações feitas conjuntamente.


**c) Arquitetura final que supera o desempenho da rede proposta no enunciado:**
- Arquitetura:
  - Uma camada convolucional com 32 kernels de dimensão 4x4, stride (1,1) e com funções de ativação ReLU.
  - Uma segunda camada convolucional com 64 kernels de dimensão 3x3, stride (1,1), com funções de ativação ReLU e taxa de ocorrência de dropout de 25%.
  - Camada do tipo Flatten
  - Uma camada de saída com 10 neurônios com função de ativação softmax (não deve ser alterada).
- Treinamento: 
  - Algoritmo adaptativo Adam, 5 épocas e 32 amostras por mini-batch (default do método fit()).
  - Função custo (loss): sparse_categorical_crossentropy (não deve ser alterada).
  - Métrica auxiliar: Acurácia (não deve ser alterada).
- Resultado do treinamento dos 3 modelos:
  - Perdas: 0.0158
  - Acurácia: 0.9949

In [0]:
# Rede proposta: 32 kernels na camada 1
num_models = 3
EPOCHS = 5
media_metricas = []

lista_metricas = { 'loss': [], 'accuracy': []}
metricas = {}

for i in range(0, num_models):
    CNN = keras.models.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:]),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Dropout(0.25),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='softmax')
    ])

    CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = CNN.fit(x_train, y_train, epochs=5)

    lista_metricas['loss'].append(history.history['loss'][-1])
    lista_metricas['accuracy'].append(history.history['accuracy'][-1])

for key in lista_metricas.keys():
    metricas[key] = sum(lista_metricas[key])/len(lista_metricas[key])

media_metricas.append(metricas)
print(media_metricas)

In [0]:
# Rede final:
num_models = 3
EPOCHS = 5
media_metricas = []

lista_metricas = { 'loss': [], 'accuracy': []}
metricas = {}

for i in range(0, num_models):
    CNN = keras.models.Sequential([
        keras.layers.Conv2D(32, kernel_size=(4, 4), activation='relu', input_shape=x_train.shape[1:]),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.Dropout(0.25),
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation='softmax')
    ])

    CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = CNN.fit(x_train, y_train, epochs=5)

    lista_metricas['loss'].append(history.history['loss'][-1])
    lista_metricas['accuracy'].append(history.history['accuracy'][-1])

for key in lista_metricas.keys():
    metricas[key] = sum(lista_metricas[key])/len(lista_metricas[key])

media_metricas.append(metricas)
print(media_metricas)

### **Parte 2 - Comparação de todos os modelos:**

- A Tabela 7 apresenta os desempenhos dos 4 classificadores estudados nas questões 1 a 4.
- Classificador Linear:
  - Coeficiente de Regularização: 965.8832
  - Critério de quadrados mínimos
- Máquina de Aprendizado Extremo:
  - Camada intermediária: 1000 neurônios com função de ativação ReLU com pesos definidos aleatoriamente de acordo com uma função de distribuição normal com média nula e desvio padrão de 0.2.
  - Camada de saída: 10 neurônios com função de ativação linear.
  - Critério de quadrados mínimos
- Redes MLP e convolucional: estruturas já apresentados neste *notebook* (item f da questão 3 e item c da questão 4, respectivamente).  

<h><center>Tabela 7: Desempenho dos 4 modelos de classificadores obtidos nos exercícios dos EFCs 1 e 2 junto aos dados de treinamento.</center></h>

| Modelo de Classificador  | Acurácia | Parâmetros ajustáveis |
|-------------------------------|----------|------------------|
|Linear                         |0.8570    |  7850            |
|Máquina de Aprendizado Extremo |0.9456    | 10010            |
|Rede MLP                       |0.9887    |407050            |
|Rede Convolucional             |0.9949    |357610            |

- Como era esperado, o classificador linear apresentou o pior desempenho dentre os modelos, devido ao número reduzido de parâmetros ajustáveis e a propriedade de gerar apenas fronteiras de decisão lineares para separação das classes.

- Na sequência está a máquina de aprendizado extremo, cujo ganho de desempenho deve-se a aplicação de funções de ativação não-lineares nos dados de entrada, tornando o modelo capaz de gerar mapeamentos (e fronteiras de decisão) não-lineares e mais flexibilidade. 
- No entanto, o desempenho da ELM é inferior ao da rede MLP pois a flexibilidade alcançada pela ELM é menor, sendo consequência do menor número de parâmetros ajustáveis.
- Já as redes MLP e convolucional estudadas nesse EFC foram capazes de superar significativamente o desempenho dos modelos anteriores. No caso da MLP foi alcançado um desempenho de 98.87%, enquanto a rede convolucional atingiu 99.49%.
- Podemos dizer que o alto desempenho da rede MLP deve-se ao elevado nível de flexibilidade do modelo devido ao seu número elevado de parâmetros ajustáveis.
- Por outro lado, o desempenho alcançado pela rede convolucional deve-se às camadas convolucionais e suas propriedades, tais como:
  - A rede convolucional não requer a vetorização das imagens de entrada.
  - Leva em conta o caráter espacial das imagens.
  - Há uma redução significativa do número de parâmetros ajustáveis, pois as camadas convolucionais realizam compartilhamento de pesos (as duas camadas convolucionais juntas possuem apenas 19040 pesos sinápticos).
  - Maior capacidade de extração de atributos pelos filtros convolucionais.



